In [1]:
from fls import *
import sys
import os
import re
from collections import namedtuple
__VERSION__ = "1.4 [fastlane]"
__DATE__ = "07/May/2023"

# Convert NBTest

Converts files `NBTest_9999_Comment.py -> test_9999_Comment.py` suitable for `pytest`

In [2]:
print(f"NBTestConvert v{__VERSION__} {__DATE__}")

NBTestConvert v1.3.1 [fastlane] 30/Apr/2023


In [3]:
NOTEST_DEFAULT="TEST"
LIBRARY = "fastlane_bot"

## Get script path and set paths

In [4]:
sys.argv[0].rsplit("/", maxsplit=1)

['/Users/skl/opt/anaconda3/lib/python3.8/site-packages',
 'ipykernel_launcher.py']

In [5]:
sys.argv[0].rsplit("/", maxsplit=1)[-1]

'ipykernel_launcher.py'

In [6]:
if sys.argv[0].rsplit("/", maxsplit=1)[-1]=="ipykernel_launcher.py":
    JUPYTER = True
    SCRIPTPATH = os.getcwd()
else:
    JUPYTER = False
    SCRIPTPATH = os.path.dirname(os.path.realpath(sys.argv[0]))

In [7]:
SRCPATH = os.path.join(SCRIPTPATH, "")
TRGPATH = os.path.join(SCRIPTPATH, f"../../{LIBRARY}/tests/nbtest")
print(TRGPATH)

/Users/skl/REPOES/Bancor/ArbBot/resources/NBTest/../../fastlane_bot/tests/nbtest


In [8]:
print("JUPYTER", JUPYTER)
print("SCRIPTPATH", SCRIPTPATH)
print("SRCPATH", SRCPATH)
print("TRGPATH", TRGPATH)
print("---")

JUPYTER True
SCRIPTPATH /Users/skl/REPOES/Bancor/ArbBot/resources/NBTest
SRCPATH /Users/skl/REPOES/Bancor/ArbBot/resources/NBTest/
TRGPATH /Users/skl/REPOES/Bancor/ArbBot/resources/NBTest/../../fastlane_bot/tests/nbtest
---


## Generate the list of files

In [9]:
rawlist = os.listdir(SRCPATH)
rawlist.sort()
rawlist

['.gitignore',
 '.ipynb_checkpoints',
 'ConvertNBTest.ipynb',
 'ConvertNBTest.py',
 'NBTest_000_Template.ipynb',
 'NBTest_000_Template.py',
 'NBTest_002_ContractHelper.ipynb',
 'NBTest_002_ContractHelper.py',
 'NBTest_003_PoolManager.ipynb',
 'NBTest_003_PoolManager.py',
 'NBTest_004_TokenManager.ipynb',
 'NBTest_004_TokenManager.py',
 'NBTest_005_AggregatCarbonTrades.ipynb',
 'NBTest_005_AggregatCarbonTrades.py',
 'NBTest_006_GetPriceMap.ipynb',
 'NBTest_006_GetPriceMap.py',
 'NBTest_007_TopNpoolsOnexchange.ipynb',
 'NBTest_007_TopNpoolsOnexchange.py',
 'NBTest_008_TxHelper.ipynb',
 'NBTest_008_TxHelper.py',
 'NBTest_063b_Optimizer.ipynb',
 'NBTest_063b_Optimizer.py',
 'SKLTesting.ipynb',
 'SKLTesting.py',
 '__pycache__',
 'carbon',
 'fastlane_bot',
 'fls.py',
 'jupytext-metadata-template.ipynb',
 'jupytext-metadata-template.py',
 '~$opt.xlsx']

In [10]:
dr_nt = namedtuple("datarecord_nt", "tid, comment, fn, outfn")
def filterfn(fn):
    """
    takes fn and returns either filelist_nt or None 
    """
    nxsplit = fn.rsplit(".", maxsplit=1)
    if len(nxsplit) < 2: return None
    if not(nxsplit[1].lower()=="py"): return None
    fnsplit = nxsplit[0].split("_")
    if not len(fnsplit) in [2,3]: return None
    if not fnsplit[0] == "NBTest": return None
    tid = fnsplit[1]
    try:
        comment = fnsplit[2]
    except IndexError:
        comment = ""
    outfn = f"test_{tid}_{comment}.py"
    return dr_nt(tid=tid, comment=comment, fn=fn, outfn=outfn)

assert filterfn("README") is None
assert filterfn("NBTest_0000_Bla.ipynb") is None
assert filterfn("NBTest_0000.py")
assert filterfn("Test_0000_Bla.py") is None
assert filterfn("NBTest_1.10.4_Bla.py").tid == "1.10.4"
assert filterfn("NBTest_1.py").comment == ""
filterfn("NBTest_0000_Bla.py")

datarecord_nt(tid='0000', comment='Bla', fn='NBTest_0000_Bla.py', outfn='test_0000_Bla.py')

In [11]:
fnlst = (filterfn(fn) for fn in rawlist)
fnlst = tuple(r for r in fnlst if not r is None)
#fnlst = (fnlst[1],)
fnlst

(datarecord_nt(tid='000', comment='Template', fn='NBTest_000_Template.py', outfn='test_000_Template.py'),
 datarecord_nt(tid='002', comment='ContractHelper', fn='NBTest_002_ContractHelper.py', outfn='test_002_ContractHelper.py'),
 datarecord_nt(tid='003', comment='PoolManager', fn='NBTest_003_PoolManager.py', outfn='test_003_PoolManager.py'),
 datarecord_nt(tid='004', comment='TokenManager', fn='NBTest_004_TokenManager.py', outfn='test_004_TokenManager.py'),
 datarecord_nt(tid='005', comment='AggregatCarbonTrades', fn='NBTest_005_AggregatCarbonTrades.py', outfn='test_005_AggregatCarbonTrades.py'),
 datarecord_nt(tid='006', comment='GetPriceMap', fn='NBTest_006_GetPriceMap.py', outfn='test_006_GetPriceMap.py'),
 datarecord_nt(tid='007', comment='TopNpoolsOnexchange', fn='NBTest_007_TopNpoolsOnexchange.py', outfn='test_007_TopNpoolsOnexchange.py'),
 datarecord_nt(tid='008', comment='TxHelper', fn='NBTest_008_TxHelper.py', outfn='test_008_TxHelper.py'),
 datarecord_nt(tid='063b', comment=

## Process files

In [12]:
def funcn(title):
    """
    converts a title into a function name
    
    NOTE
    
    "This is a title [TEST]"     -> test_this_is_a_title
    "This is a title [NOTEST]"   -> notest_this_is_a_title
    "This is a title"            -> depends on NOTEST_DEFAULT global
    """
    global NOTEST_DEFAULT
    #print("[funcn] NOTEST_DEFAULT", NOTEST_DEFAULT)
    
    title = title.strip()
    if title[-8:] == "[NOTEST]":
        notest = True
        title = title[:-8].strip()
    elif title[-6:] == "[TEST]":
        notest = False
        title = title[:-6].strip()
    else:
        notest = True if NOTEST_DEFAULT == "NOTEST" else False 
        
        
    prefix = "notest_" if notest else "test_"

        
    funcn = title.lower()
    funcn = funcn.replace(" ", "_")
    funcn = prefix+funcn
    return funcn

assert funcn(" Title [TEST]  ") == "test_title"
assert funcn(" Title [NOTEST] ") == "notest_title"
assert funcn(" Title  ") == "notest_title" if NOTEST_DEFAULT=="NOTEST" else "test_title"
assert funcn(" Advanced Testing [TEST]  ") == "test_advanced_testing"
assert funcn(" A notest title [NOTEST] ") == "notest_a_notest_title"
#funcn("Asserting that the radius computes correctly")

In [13]:
funcn("A notest title [NOTEST]")

'notest_a_notest_title'

In [14]:
def process_code(code, dr, srcpath=None, trgpath=None):
    """
    processes notebook code
    
    :code:      the code to be processed
    :dr:        the associated data record (datarecord_nt)
    :srcpath:   source path (info only)
    :trgpath:   target path (info only)
    """
    lines = code.splitlines()
    outlines = [
                 "# "+"-"*60,
                f"# Auto generated test file `{dr.outfn}`",
                 "# "+"-"*60,
                f"# source file   = {dr.fn}"
    ]
#     if srcpath and srcpath != ".":
#         outlines += [
#                 f"# source path   = {srcpath}"
#         ]
#     if trgpath and trgpath != ".":
#         outlines += [
#                 f"# target path   = {srcpath}"
#         ]
    outlines += [
        
                f"# test id       = {dr.tid}",
                f"# test comment  = {dr.comment}",
                 "# "+"-"*60,
                "","",
    ]
    is_precode = True
    for l in lines:
#         print(l)
#         try:
#             print(l[:5], l[:5].encode(), ord(l[1]), ord(l[4]), l[:5]=="# ## ")
#         except:
#             pass
        
        if l[:4] == "# # ":
            print(f"""Processing "{l[4:]}" ({r.fn})""")
            outlines += [""]
            
        elif l[:5] == "# ## " or l[:5].encode() == b'# ##\xc2\xa0':
            title = l[5:].strip()
            fcn = funcn(title)
            print(f"  creating function `{fcn}()` from section {title}")
            outlines += [
                 "",
                 "# "+"-"*60,
                f"# Test      {r.tid}",
                f"# File      {r.outfn}",
                f"# Segment   {title}",
                 "# "+"-"*60,
                f"def {fcn}():",
                 "# "+"-"*60,
            ]
            is_precode = False
            
        elif l[:9] == "# NBTEST:":
            l = l[9:]
            try:
                opt, val = l.split("=")
                opt=opt.strip().upper()
                val=val.strip().upper()
            except:
                print(f"  error setting option", l)
                raise ValueError("Error setting option", l, dr.fn)
            print(f"  processiong option {opt}={val}")
            if opt == "NOTEST_DEFAULT":
                global NOTEST_DEFAULT
                if val in ["TEST", "NOTEST"]:
                    NOTEST_DEFAULT = val
                    #print("[process_code] NOTEST_DEFAULT", NOTEST_DEFAULT)
                else:
                    raise ValueError(f"Invalid choice for option NOTEST_DEFAULT: {val}", l, dr.fn)
            else:
                raise ValueError(f"Unknown option {opt}", l, dr.fn)
            
            
        else:
            if is_precode:
                if l[:2] != "# ":
                    outlines += [l]
            else:
                outlines += ["    "+l]
    outcode = "\n".join(outlines)
    return outcode

In [15]:
for r in fnlst:
    code = fload(r.fn, SRCPATH, quiet=True)
    testcode = process_code(code, r, SRCPATH, TRGPATH)
    fsave(testcode, r.outfn, TRGPATH, quiet=True)
    print(f"  saving generated test to {r.outfn}")

Processing "TEMPLATE [NBTest000]" (NBTest_000_Template.py)
  creating function `notest_demo_section()` from section Demo section [NOTEST]
  creating function `test_section_1()` from section Section 1
  creating function `test_section_2()` from section Section 2
  saving generated test to test_000_Template.py
Processing "Unit tests for ContractHelper" (NBTest_002_ContractHelper.py)
  saving generated test to test_002_ContractHelper.py
Processing "Unit tests for PoolManager" (NBTest_003_PoolManager.py)
  saving generated test to test_003_PoolManager.py
Processing "Unit tests for TokenManager" (NBTest_004_TokenManager.py)
  saving generated test to test_004_TokenManager.py
  saving generated test to test_005_AggregatCarbonTrades.py
  saving generated test to test_006_GetPriceMap.py
  saving generated test to test_007_TopNpoolsOnexchange.py
  saving generated test to test_008_TxHelper.py
Processing "CPC and Optimizer in Fastlane [NBTest063b]" (NBTest_063b_Optimizer.py)
  creating function 